In [2]:
#https://github.com/indohito/EE399/tree/main/EE399HW5
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from scipy import integrate

# Define the neural network architecture
class FFNN(nn.Module):
    def __init__(self, input_size=3, hidden_size=128, output_size=3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(FFNN().parameters(), lr=0.01)

# Define the training loop
def train(model, dataloader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        loss_record = []
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            loss_record.append(loss.item())
            if epoch % 10 == 0:
                print(f"Epoch {epoch} Loss: {loss.item():.4f}")

dt = 0.01
T = 8
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho =10
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]
        


for rho in [10, 28, 40]:
    # Generate the training data
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Train the model
    model = FFNN()
    print(f"Train for rho {rho}")
    train(model, dataloader, criterion, optimizer)
    print(" ")
    print(" ")
# Use the trained model for future state prediction for ρ = 17 and ρ = 35
for rho in [17, 35]:
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    model.eval()
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            test_loss = criterion(outputs, targets)
        print(f'Test_Loss for {rho}: {test_loss.item():.4f}')

Train for rho 10
Epoch 0 Loss: 43.4697
Epoch 0 Loss: 42.3551
Epoch 0 Loss: 43.6862
Epoch 0 Loss: 35.8543
Epoch 10 Loss: 43.5300
Epoch 10 Loss: 41.1086
Epoch 10 Loss: 43.6283
Epoch 10 Loss: 45.8069
Epoch 20 Loss: 42.9457
Epoch 20 Loss: 41.4997
Epoch 20 Loss: 44.1599
Epoch 20 Loss: 43.0997
Epoch 30 Loss: 42.0032
Epoch 30 Loss: 44.8053
Epoch 30 Loss: 41.3476
Epoch 30 Loss: 46.6941
Epoch 40 Loss: 41.4990
Epoch 40 Loss: 42.5719
Epoch 40 Loss: 43.0363
Epoch 40 Loss: 55.0838
Epoch 50 Loss: 45.5434
Epoch 50 Loss: 42.8108
Epoch 50 Loss: 40.2377
Epoch 50 Loss: 43.2071
Epoch 60 Loss: 42.6618
Epoch 60 Loss: 41.7803
Epoch 60 Loss: 43.5765
Epoch 60 Loss: 47.7935
Epoch 70 Loss: 40.1449
Epoch 70 Loss: 43.9480
Epoch 70 Loss: 43.6274
Epoch 70 Loss: 50.1799
Epoch 80 Loss: 42.6227
Epoch 80 Loss: 44.1210
Epoch 80 Loss: 41.1488
Epoch 80 Loss: 48.8016
Epoch 90 Loss: 44.0885
Epoch 90 Loss: 43.0969
Epoch 90 Loss: 42.0763
Epoch 90 Loss: 37.8485
 
 
Train for rho 28
Epoch 0 Loss: 314.2322
Epoch 0 Loss: 309.3161


In [26]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=3, hidden_size=50, num_layers=3)
        self.fc1 = nn.Linear(in_features=50, out_features=3)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc1(x)

        return x
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer_lstm = optim.Adam(LSTM().parameters(), lr=0.0001)

for rho in [10, 28, 40]:
    # Generate the training data
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Train the model
    model = LSTM()
    print(f"Train for rho {rho}")
    train(model, dataloader, criterion, optimizer_lstm)
    print(" ")
    print(" ")
# Use the trained model for future state prediction for ρ = 17 and ρ = 35
for rho in [17, 35]:
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    model.eval()
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            test_loss = criterion(outputs, targets)
        print(f'Test_Loss for {rho}: {test_loss.item():.4f}')

Train for rho 10
Epoch 0 Loss: 44.4304
Epoch 0 Loss: 44.6278
Epoch 0 Loss: 45.5824
Epoch 0 Loss: 45.4888
Epoch 10 Loss: 45.1564
Epoch 10 Loss: 44.8943
Epoch 10 Loss: 44.4646
Epoch 10 Loss: 46.3868
Epoch 20 Loss: 45.4672
Epoch 20 Loss: 44.3075
Epoch 20 Loss: 44.7701
Epoch 20 Loss: 46.0241
Epoch 30 Loss: 44.7416
Epoch 30 Loss: 45.4595
Epoch 30 Loss: 44.4589
Epoch 30 Loss: 45.3312
Epoch 40 Loss: 44.6869
Epoch 40 Loss: 44.6073
Epoch 40 Loss: 45.2477
Epoch 40 Loss: 46.1613
Epoch 50 Loss: 44.8667
Epoch 50 Loss: 44.8879
Epoch 50 Loss: 44.6814
Epoch 50 Loss: 47.0288
Epoch 60 Loss: 44.6303
Epoch 60 Loss: 45.1631
Epoch 60 Loss: 45.1351
Epoch 60 Loss: 43.1314
Epoch 70 Loss: 44.9580
Epoch 70 Loss: 44.3382
Epoch 70 Loss: 45.7687
Epoch 70 Loss: 42.0456
Epoch 80 Loss: 45.3974
Epoch 80 Loss: 44.7592
Epoch 80 Loss: 44.4127
Epoch 80 Loss: 45.9768
Epoch 90 Loss: 45.4632
Epoch 90 Loss: 44.1198
Epoch 90 Loss: 45.1004
Epoch 90 Loss: 44.9254
 
 
Train for rho 28
Epoch 0 Loss: 279.4544
Epoch 0 Loss: 280.2749


In [27]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=3, hidden_size=60, num_layers=3)
        self.fc1 = nn.Linear(in_features=60, out_features=3)
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc1(x)

        return x
    
criterion = nn.MSELoss()
optimizer_rnn = optim.Adam(RNN().parameters(), lr=0.01)

for rho in [10, 28, 40]:
    # Generate the training data
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Train the model
    model = RNN()
    print(f"Train for rho {rho}")
    train(model, dataloader, criterion, optimizer_rnn)
    print(" ")
    print(" ")
# Use the trained model for future state prediction for ρ = 17 and ρ = 35
for rho in [17, 35]:
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    model.eval()
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            test_loss = criterion(outputs, targets)
        print(f'Test_Loss for {rho}: {test_loss.item():.4f}')

Train for rho 10
Epoch 0 Loss: 44.1994
Epoch 0 Loss: 44.5685
Epoch 0 Loss: 44.8176
Epoch 0 Loss: 43.0061
Epoch 10 Loss: 44.6095
Epoch 10 Loss: 44.3365
Epoch 10 Loss: 44.6395
Epoch 10 Loss: 42.8961
Epoch 20 Loss: 44.2318
Epoch 20 Loss: 44.6378
Epoch 20 Loss: 44.2247
Epoch 20 Loss: 47.0013
Epoch 30 Loss: 44.7016
Epoch 30 Loss: 43.5682
Epoch 30 Loss: 45.3113
Epoch 30 Loss: 43.4281
Epoch 40 Loss: 44.2555
Epoch 40 Loss: 45.1700
Epoch 40 Loss: 44.0624
Epoch 40 Loss: 43.6813
Epoch 50 Loss: 44.0337
Epoch 50 Loss: 44.5998
Epoch 50 Loss: 44.7982
Epoch 50 Loss: 44.1622
Epoch 60 Loss: 44.4345
Epoch 60 Loss: 44.3034
Epoch 60 Loss: 44.4775
Epoch 60 Loss: 45.7818
Epoch 70 Loss: 44.3964
Epoch 70 Loss: 44.2081
Epoch 70 Loss: 45.0556
Epoch 70 Loss: 42.9063
Epoch 80 Loss: 44.9155
Epoch 80 Loss: 44.5275
Epoch 80 Loss: 44.0376
Epoch 80 Loss: 43.5440
Epoch 90 Loss: 44.2817
Epoch 90 Loss: 44.3503
Epoch 90 Loss: 44.8280
Epoch 90 Loss: 44.0628
 
 
Train for rho 28
Epoch 0 Loss: 278.0729
Epoch 0 Loss: 278.0535


In [ ]:

class Reservoir(nn.Module):
    def __init__(self, hidden_dim, connectivity):
        super().__init__()
    
        self.Wx = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
        self.Wh = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
        self.Uh = nn.Parameter(torch.randn(hidden_dim, hidden_dim))
        self.register_buffer('Wx_mask', (torch.rand(hidden_dim, hidden_dim) < connectivity).float())
        self.register_buffer('Wh_mask', (torch.rand(hidden_dim, hidden_dim) < connectivity).float())
        self.register_buffer('Uh_mask', (torch.rand(hidden_dim, hidden_dim) < connectivity).float())
        self.act = nn.Tanh()

    def forward(self, x, h):
        h = self.act(torch.mm(h, self.Uh*self.Uh_mask) + torch.mm(x, self.Wh*self.Wh_mask))
        y = self.act(torch.mm(h, self.Wx*self.Wx_mask))
        return y, h
    
class ESN(nn.Module):
    def __init__(self, in_dim=3, out_dim=3, reservoir_dim=100, connectivity=.6):
        super().__init__()
        self.reservoir_dim = reservoir_dim
        self.input_to_reservoir = nn.Linear(in_dim, reservoir_dim)
        self.input_to_reservoir.requires_grad_(False)
        self.reservoir = Reservoir(reservoir_dim, connectivity)
        self.readout = nn.Linear(reservoir_dim, out_dim)
  
    def forward(self, x):
        reservoir_in = self.input_to_reservoir(x)
        h = torch.ones(x.size(0), self.reservoir_dim)
        reservoirs = []
        for i in range(x.size(1)):
            out, h = self.reservoir(reservoir_in[:, i, :], h)
            reservoirs.append(out.unsqueeze(1))
        reservoirs = torch.cat(reservoirs, dim=1)
        outputs = self.readout(reservoirs)
        return outputs

criterion = nn.MSELoss()
optimizer_esn = optim.SGD(ESN().parameters(), lr=0.001)

for rho in [10, 28, 40]:
    # Generate the training data
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Train the model
    model = ESN()
    print(f"Train for rho {rho}")
    train(model, dataloader, criterion, optimizer_esn)
    print(" ")
    print(" ")
    
# Use the trained model for future state prediction for ρ = 17 and ρ = 35
for rho in [17, 35]:
    x0 = -15 + 30 * np.random.random((100, 3))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho))
                      for x0_j in x0])
    nn_input = torch.tensor(x_t[:,:-1,:], dtype=torch.float32)
    nn_output = torch.tensor(x_t[:,1:,:], dtype=torch.float32)
    dataset = TensorDataset(nn_input, nn_output)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    model.eval()
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            test_loss = criterion(outputs, targets)
        print(f'Test_Loss for {rho}: {test_loss.item():.4f}')


Train for rho 10
Epoch 0 Loss: 45.0758
Epoch 0 Loss: 44.5551
Epoch 0 Loss: 44.8880
Epoch 0 Loss: 45.7044
Epoch 10 Loss: 44.3023
Epoch 10 Loss: 44.9313
Epoch 10 Loss: 45.1238
Epoch 10 Loss: 46.9824
Epoch 20 Loss: 44.7238
Epoch 20 Loss: 44.7148
Epoch 20 Loss: 45.2225
Epoch 20 Loss: 44.5666
Epoch 30 Loss: 44.9784
Epoch 30 Loss: 44.7087
Epoch 30 Loss: 44.6958
Epoch 30 Loss: 46.7870
Epoch 40 Loss: 45.0413
Epoch 40 Loss: 44.5800
Epoch 40 Loss: 45.0185
Epoch 40 Loss: 44.7328
Epoch 50 Loss: 45.1211
Epoch 50 Loss: 44.5101
Epoch 50 Loss: 44.9537
Epoch 50 Loss: 45.1677
Epoch 60 Loss: 45.0223
Epoch 60 Loss: 44.2101
Epoch 60 Loss: 44.9057
Epoch 60 Loss: 48.7551
Epoch 70 Loss: 45.0062
Epoch 70 Loss: 44.7421
Epoch 70 Loss: 44.8358
Epoch 70 Loss: 45.1817
